## Sources

In [13]:
import pandas as pd
import numpy as np
import csv
import uuid
from datetime import datetime

In [14]:
ROOT = "/workspaces/ontologie-peuplement/"  #/home/STual/KG-cadastre/
PATH = ROOT + "data/gentilly/SOURCES.csv"
OUTPUT_PATH = ROOT + "data/rdf/"

sources = pd.read_csv(PATH,header=0)
display(sources)

,CODE_ARCHIVES,rdf:type,rico:name,rico:location,rico:identifier,rsfs:comment/Folios,isSourceType,hasCreationDate,hasValidTime/hasBeginning,hasValidTime/hasEnd,rico:isOrWasIncludedIn,rico:isOrWasInstantiationOf,rico:isOrWasDigitalInstantiationOf,rico:hasOrHadDerivedInstanciation,rico:hasOrHadPhysicalLocation
0,94,RecordSet,Matrices des propriétés bâties et non bâties,Gentilly,MAT_B_NB_1813,NaN,MatriceCadastrale1807_1822_Bati_NonBati,1813.0,1813.0,1835.0,CAD1,NaN,NaN,NaN,NaN
1,94,Instanciation,NaN,Gentilly,3P255,NB : 1-542 ; B : 1-166,NaN,NaN,NaN,NaN,NaN,MAT_B_NB_1813,NaN,FRAD094_3P_000255_01,Archives Départementales du Val-de-Marne (94)
2,94,Instanciation,NaN,Gentilly,FRAD094_3P_000255_01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MAT_B_NB_1813,NaN,NaN
3,94,RecordSet,Matrice des propriétés foncières (1836-1848),Gentilly,MAT_NB_1836,NaN,MatriceCadastrale1822_1914_NonBati,1836.0,1836.0,1835.0,CAD1,NaN,NaN,NaN,NaN
4,94,Instanciation,NaN,Gentilly,3P256,1-480,NaN,NaN,NaN,NaN,NaN,MAT_NB_1836,NaN,FRAD094_3P_000256_01,Archives Départementales du Val-de-Marne (94)
5,94,Instanciation,NaN,Gentilly,3P257,481-980,NaN,NaN,NaN,NaN,NaN,MAT_NB_1836,NaN,FRAD094_3P_000257_01,Archives Départementales du Val-de-Marne (94)
6,94,Instanciation,NaN,Gentilly,3P258,981-1150,NaN,NaN,NaN,NaN,NaN,MAT_NB_1836,NaN,FRAD094_3P_000258_01,Archives Départementales du Val-de-Marne (94)
7,94,Instanciation,NaN,Gentilly,FRAD094_3P_000256_01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MAT_NB_1836,NaN,NaN
8,94,Instanciation,NaN,Gentilly,FRAD094_3P_000257_01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MAT_NB_1836,NaN,NaN
9,94,Instanciation,NaN,Gentilly,FRAD094_3P_000258_01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MAT_NB_1836,NaN,NaN


In [15]:
# Print the list of column headers
print(sources.columns.tolist())

['CODE_ARCHIVES', 'rdf:type', 'rico:name', 'rico:location', 'rico:identifier', 'rsfs:comment/Folios', 'isSourceType', 'hasCreationDate', 'hasValidTime/hasBeginning', 'hasValidTime/hasEnd', 'rico:isOrWasIncludedIn', 'rico:isOrWasInstantiationOf', 'rico:isOrWasDigitalInstantiationOf', 'rico:hasOrHadDerivedInstanciation', 'rico:hasOrHadPhysicalLocation']


In [16]:
from rdflib import Graph, Literal, Namespace, RDF, URIRef, BNode
from rdflib.namespace import XSD
from namespaces import *

# Create a new RDF graph
g = Graph()

g.bind('source', srcuri)
g.bind('cad', cad)
g.bind('add', add)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)
g.bind('srctype', srctype)
g.bind('mlclasse', mlclasse)

# Iterate over each row in the DataFrame
for index, row in sources.iterrows():
    CODE_ARCHIVES = str(int(row['CODE_ARCHIVES']))

    #URI = http://data.ign.fr/id/sources/{location}_{identifier}
    subject_uri = URIRef(srcuri + f"{CODE_ARCHIVES}_{str(row['rico:location'])}_{str(row['rico:identifier'])}")
    g.add((subject_uri, rico.identifier, Literal(row["rico:identifier"])))

    if not pd.isnull(row["rico:name"]):
        g.add((subject_uri, rico.name, Literal(row["rico:name"])))
    if not pd.isnull(row["isSourceType"]):
        g.add((subject_uri, cad.isSourceType, URIRef(srctype + f'{row["isSourceType"]}')))
        
    ##Define properties if object is a RecordResource
    if row["rdf:type"] == "RecordSet" or row["rdf:type"] == "Record":
        g.add((subject_uri, RDF.type, rico.RecordSet))
        g.add((subject_uri, rico.location, Literal(row["rico:location"])))
        
        if not pd.isnull(row["rico:isOrWasIncludedIn"]):
            g.add((subject_uri, rico.isOrWasIncludedIn, URIRef(srcuri + f"{CODE_ARCHIVES}_{str(row['rico:location'])}_{str(row['rico:isOrWasIncludedIn'])}")))
        ## Date de création
        creationDateBNode = BNode()
        g.add((subject_uri, rico.hasCreationDate, URIRef(creationDateBNode.n3())))
        g.add((URIRef(creationDateBNode.n3()), RDF.type, add.CrispTimeInstant))
        g.add((URIRef(creationDateBNode.n3()), add.timeCalendar, time.Gregorian))
        g.add((URIRef(creationDateBNode.n3()), add.timePrecision, time.Year))
        str_date = datetime.strptime(str(int(row["hasCreationDate"])) + '-01-01', '%Y-%m-%d')
        g.add((URIRef(creationDateBNode.n3()), add.timeStamp, Literal(str_date, datatype=XSD.dateTimeStamp)))
        ## Temps valide
        validTimeNode = BNode()
        ### Début de validité
        validTimeNodeBegin = BNode()
        g.add((subject_uri, add.hasTime, URIRef(validTimeNode.n3())))
        g.add((URIRef(validTimeNode.n3()), RDF.type, time.Interval))
        g.add((URIRef(validTimeNode.n3()), add.hasBeginning, URIRef(validTimeNodeBegin.n3())))
        g.add((URIRef(validTimeNodeBegin.n3()), RDF.type, add.TimeInstant))
        g.add((URIRef(validTimeNodeBegin.n3()), add.timeCalendar, time.Gregorian))
        g.add((URIRef(validTimeNodeBegin.n3()), add.timePrecision, time.Year))
        str_date = datetime.strptime(str(int(row["hasValidTime/hasBeginning"])) + '-01-01', '%Y-%m-%d')
        g.add((URIRef(validTimeNodeBegin.n3()), add.timeStamp, Literal(str_date, datatype=XSD.dateTimeStamp)))
        ### Fin de validité
        validTimeNodeEnd = BNode()
        g.add((URIRef(validTimeNode.n3()), add.hasEnd, URIRef(validTimeNodeEnd.n3())))
        g.add((URIRef(validTimeNodeEnd.n3()), add.timeCalendar, time.Gregorian))
        g.add((URIRef(validTimeNodeEnd.n3()), add.timePrecision, time.Year))
        str_date = datetime.strptime(str(int(row["hasValidTime/hasEnd"])) + '-12-31', '%Y-%m-%d')
        g.add((URIRef(validTimeNodeEnd.n3()), add.timeStamp, Literal(str_date, datatype=XSD.dateTimeStamp)))

    #else if Instanciation
    elif row["rdf:type"] == "Instanciation":
        if row["rico:identifier"][0:2] == '3P' or row["rico:identifier"][0:2] == 'CN':
            g.add((subject_uri, RDF.type, rico.Instantiation))
            g.add((subject_uri,rico.isOrWasInstantiationOf,URIRef(srcuri + f'{CODE_ARCHIVES}_{row["rico:location"]}_{row["rico:isOrWasInstantiationOf"]}')))
            g.add((subject_uri, rico.hasOrHadDerivedInstanciation, URIRef(srcuri + f'{CODE_ARCHIVES}_{row["rico:location"]}_{row["rico:hasOrHadDerivedInstanciation"]}')))
            g.add((subject_uri,rico.hasOrHadPhysicalLocation,Literal(row['rico:hasOrHadPhysicalLocation'])))
        elif row["rico:identifier"][0:2] != '3P' or row["rico:identifier"][0:2] != 'CN' :
            g.add((subject_uri, RDF.type, rico.Instantiation))
            g.add((subject_uri,rico.isOrWasDigitalInstantiationOf,URIRef(srcuri + f'{CODE_ARCHIVES}_{row["rico:location"]}_{row["rico:isOrWasDigitalInstantiationOf"]}')))

# Print out the graph in Turtle syntax
print(g.serialize(format='turtle'))
#write the graph to a file
g.serialize(destination=f'{OUTPUT_PATH}/sources.ttl', format='turtle')

@prefix add: <http://rdf.geohistoricaldata.org/def/address#> .
@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix rico: <https://www.ica.org/standards/RiC/ontology#> .
@prefix source: <http://data.ign.fr/id/source/> .
@prefix srctype: <http://data.ign.fr/id/codes/cadastre/sourceType/> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

source:75_Gentilly_CN80 a rico:Instantiation ;
    cad:isSourceType srctype:PlanParcellaireFinal ;
    rico:hasOrHadDerivedInstanciation source:75_Gentilly_CN80_digit ;
    rico:hasOrHadPhysicalLocation "Archives de Paris" ;
    rico:identifier "CN80" ;
    rico:isOrWasInstantiationOf source:75_Gentilly_PLAN_SD_1845 ;
    rico:name "Plan révisé - Section D dite de la Barrière de Fontainebleau" .

source:75_Gentilly_digit_CN80 a rico:Instantiation ;
    rico:identifier "digit_CN80" ;
    rico:isOrWasDigitalInstantiationOf source:75_Gentilly_PLAN_SD_1845 .

source:94_Gentilly_3P255 a rico:Instantiatio

<Graph identifier=N4a2217ad49944c6fa3ba09c0d399866b (<class 'rdflib.graph.Graph'>)>